In [2]:
# etl_transform.py
# This script applies at least four meaningful transformations to raw_data.csv and incremental_data.csv,
# saves the results as transformed_full.csv and transformed_incremental.csv, and includes a visualization.

import pandas as pd
import os
import matplotlib.pyplot as plt

# Create data directory and save raw data if not already present
data_path = 'data'
if os.path.exists(data_path) and not os.path.isdir(data_path):
    print(f"Error: '{data_path}' exists as a file, not a directory. Please remove or rename the file.")
    raise FileExistsError(f"Cannot create directory '{data_path}' because a file with that name exists.")
os.makedirs(data_path, exist_ok=True)

# Create raw_data.csv and incremental_data.csv (using provided data structure)
raw_data = pd.DataFrame({
    'order_id': [1, 2, 3, 4, 5, 4, 7, 8, 9, 10],  # Truncated for brevity, replace with full 100 rows if needed
    'customer_name': ['Diana', 'Eve', 'Charlie', 'Eve', 'Eve', 'Eve', 'Charlie', 'Charlie', 'Charlie', 'Eve'],
    'product': ['Tablet', 'Laptop', 'Laptop', 'Laptop', 'Tablet', 'Laptop', 'Monitor', 'Laptop', 'Monitor', 'Monitor'],
    'quantity': [None, None, 2, 2, 3, 2, 2, 3, None, 1],
    'unit_price': [500, None, 250, 750, None, 750, 750, None, 750, 500],
    'order_date': ['1/20/2024', '4/29/2024', '1/8/2024', '1/7/2024', '3/7/2024', '1/7/2024', '2/2/2024', '2/17/2024', '3/16/2024', '2/28/2024'],
    'region': ['South', 'North', None, 'West', 'South', 'West', 'West', None, 'West', 'North']
})
incremental_data = pd.DataFrame({
    'order_id': [101, 102, 103, 104, 105, 106, 107, 108, 109, 110],
    'customer_name': ['Alice', None, None, None, 'Heidi', None, None, None, 'Grace', 'Heidi'],
    'product': ['Laptop', 'Laptop', 'Laptop', 'Tablet', 'Tablet', 'Laptop', 'Tablet', 'Laptop', 'Laptop', 'Phone'],
    'quantity': [None, 1, 1, None, 2, 2, 1, None, 2, None],
    'unit_price': [900, 300, 600, 300, 600, 600, 600, 600, 600, None],
    'order_date': ['5/9/2024', '5/7/2024', '5/4/2024', '5/26/2024', '5/21/2024', '5/18/2024', '5/13/2024', '5/11/2024', '5/29/2024', '5/24/2024'],
    'region': ['Central', 'Central', 'Central', 'Central', 'North', 'Central', 'Central', None, 'Central', None]
})

# Save raw data to CSV
raw_data.to_csv('data/raw_data.csv', index=False)
incremental_data.to_csv('data/incremental_data.csv', index=False)
print("CSV files created in data/ folder")

# Create transformed directory if it doesn't exist
transformed_path = 'data/transformed'
if os.path.exists(transformed_path) and not os.path.isdir(transformed_path):
    print(f"Error: '{transformed_path}' exists as a file, not a directory. Please remove or rename the file.")
    raise FileExistsError(f"Cannot create directory '{transformed_path}' because a file with that name exists.")
os.makedirs(transformed_path, exist_ok=True)

# Load raw data
raw_data = pd.read_csv('data/raw_data.csv')
incremental_data = pd.read_csv('data/incremental_data.csv')

# Function to display before and after
def show_before_after(df_before, df_after, transformation_name):
    print(f'\nBefore {transformation_name}:')
    print(df_before.head())
    print(f'\nAfter {transformation_name}:')
    print(df_after.head())

# Transformation 1: Cleaning - Handle missing values
print('Transformation 1: Handling missing values')
raw_data_cleaned = raw_data.copy()
incremental_data_cleaned = incremental_data.copy()

# Fill missing customer_name with 'Unknown'
raw_data_cleaned['customer_name'] = raw_data_cleaned['customer_name'].fillna('Unknown')
incremental_data_cleaned['customer_name'] = incremental_data_cleaned['customer_name'].fillna('Unknown')

# Fill missing quantity with median
raw_data_cleaned['quantity'] = raw_data_cleaned['quantity'].fillna(raw_data_cleaned['quantity'].median())
incremental_data_cleaned['quantity'] = incremental_data_cleaned['quantity'].fillna(incremental_data_cleaned['quantity'].median())

# Fill missing unit_price with median
raw_data_cleaned['unit_price'] = raw_data_cleaned['unit_price'].fillna(raw_data_cleaned['unit_price'].median())
incremental_data_cleaned['unit_price'] = incremental_data_cleaned['unit_price'].fillna(incremental_data_cleaned['unit_price'].median())

# Fill missing region with 'Unknown'
raw_data_cleaned['region'] = raw_data_cleaned['region'].fillna('Unknown')
incremental_data_cleaned['region'] = incremental_data_cleaned['region'].fillna('Unknown')

# Fill missing order_date with a default date
raw_data_cleaned['order_date'] = raw_data_cleaned['order_date'].fillna('1/1/2024')
incremental_data_cleaned['order_date'] = incremental_data_cleaned['order_date'].fillna('5/1/2024')

show_before_after(raw_data, raw_data_cleaned, 'Cleaning - Missing Values')
show_before_after(incremental_data, incremental_data_cleaned, 'Cleaning - Missing Values')
print('Why: Missing values can skew analysis. Imputed customer_name and region with \'Unknown\', quantity and unit_price with median, and order_date with a default date to maintain data integrity.')

# Transformation 2: Cleaning - Remove duplicates
print('\nTransformation 2: Removing duplicates')
raw_data_no_duplicates = raw_data_cleaned.drop_duplicates()
incremental_data_no_duplicates = incremental_data_cleaned.drop_duplicates()

show_before_after(raw_data_cleaned, raw_data_no_duplicates, 'Removing Duplicates')
show_before_after(incremental_data_cleaned, incremental_data_no_duplicates, 'Removing Duplicates')
print(f'Why: Removed {len(raw_data_cleaned) - len(raw_data_no_duplicates)} duplicates from raw_data and {len(incremental_data_cleaned) - len(incremental_data_no_duplicates)} from incremental_data to ensure unique records.')

# Transformation 3: Enrichment - Add total_price column
print('\nTransformation 3: Adding total_price column')
raw_data_enriched = raw_data_no_duplicates.copy()
incremental_data_enriched = incremental_data_no_duplicates.copy()

raw_data_enriched['total_price'] = raw_data_enriched['quantity'] * raw_data_enriched['unit_price']
incremental_data_enriched['total_price'] = incremental_data_enriched['quantity'] * incremental_data_enriched['unit_price']

show_before_after(raw_data_no_duplicates, raw_data_enriched, 'Adding total_price')
show_before_after(incremental_data_no_duplicates, incremental_data_enriched, 'Adding total_price')
print('Why: Added total_price (quantity * unit_price) to enable analysis of order value.')

# Transformation 4: Structural - Convert order_date to datetime and extract year
print('\nTransformation 4: Converting order_date to datetime and extracting year')
raw_data_structured = raw_data_enriched.copy()
incremental_data_structured = incremental_data_enriched.copy()

# Convert order_date to datetime, handling MM/DD/YYYY format
raw_data_structured['order_date'] = pd.to_datetime(raw_data_structured['order_date'], format='%m/%d/%Y')
incremental_data_structured['order_date'] = pd.to_datetime(incremental_data_structured['order_date'], format='%m/%d/%Y')

raw_data_structured['order_year'] = raw_data_structured['order_date'].dt.year
incremental_data_structured['order_year'] = incremental_data_structured['order_date'].dt.year

show_before_after(raw_data_enriched, raw_data_structured, 'Converting order_date')
show_before_after(incremental_data_enriched, incremental_data_structured, 'Converting order_date')
print('Why: Converted order_date to datetime with MM/DD/YYYY format for consistency and extracted year for time-based analysis.')

# Save transformed files
raw_data_structured.to_csv('data/transformed/transformed_full.csv', index=False)
incremental_data_structured.to_csv('data/transformed/transformed_incremental.csv', index=False)
print('\nTransformed files saved to data/transformed/ directory.')

# Bonus: Visualization - Bar chart of total sales by product
print('\nBonus: Visualizing total sales by product')
total_sales = raw_data_structured.groupby('product')['total_price'].sum().reset_index()
plt.figure(figsize=(8, 6))
plt.bar(total_sales['product'], total_sales['total_price'], color='skyblue')
plt.title('Total Sales by Product')
plt.xlabel('Product')
plt.ylabel('Total Sales ($)')
plt.savefig('total_sales_by_product.png')
plt.close()
print('Why: Visualized total sales by product to identify top-performing products. Saved as total_sales_by_product.png.')

CSV files created in data/ folder
Transformation 1: Handling missing values

Before Cleaning - Missing Values:
   order_id customer_name product  quantity  unit_price order_date region
0         1         Diana  Tablet       NaN       500.0  1/20/2024  South
1         2           Eve  Laptop       NaN         NaN  4/29/2024  North
2         3       Charlie  Laptop       2.0       250.0   1/8/2024    NaN
3         4           Eve  Laptop       2.0       750.0   1/7/2024   West
4         5           Eve  Tablet       3.0         NaN   3/7/2024  South

After Cleaning - Missing Values:
   order_id customer_name product  quantity  unit_price order_date   region
0         1         Diana  Tablet       2.0       500.0  1/20/2024    South
1         2           Eve  Laptop       2.0       750.0  4/29/2024    North
2         3       Charlie  Laptop       2.0       250.0   1/8/2024  Unknown
3         4           Eve  Laptop       2.0       750.0   1/7/2024     West
4         5           Eve  Tabl